# Heroes of Pymoli Analysis

### Summary
* Across genders, purchase count and total revenue roughly match the demographic split. 
    * Males comprise 84.03% of the user population, completed 83.59% of the orders that amounted to 82% of the total revenue.
    * Females comprise 14.06% of the user population, completed 14.49% of the purchases and generated 15.21% revenue.
    * Users that identified as Other/Non-disclosed comprise 1.91% of the user population, completed 1.92% purchases and generated 2.11% of total revenue.
    * The average non-male user spent 12.5% more in total than males, which indicates an opportunity for the gaming company to capture more revenue by increasing the purchase count from each of those two gender groups.
    
* Across age groups, you can observe similar patterns wherein the demographic split is very similar to the purchase count and revenue split.
    * 20-24 year-olds are the dominant consumers, as they are the most plentiful and highest spending age group, accounting for 44.79% of the population and generated 46.85% of total revenue.
    * It is interesting that the 25-29 age group spent less on each purchase compared to the 30-34 year old group, and could be a valuable opportunity for companies to capture more value. They also had the second lowest average total purchase, even less than 15-19 age groups.
    * The <10 y.o age group surprisingly spent the second most on average per user, more than most adults.


In [2]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
file_to_load = "Resources/purchase_data.csv"

# Read Purchasing File and store into Pandas data frame
purchase_data = pd.read_csv(file_to_load)

In [3]:
purchase_data.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


### Player Count

* Total Number of Players

In [4]:
total_count= purchase_data['SN'].nunique()
data = {"Total Player Count":total_count}
count_df = pd.DataFrame(data,index=[0])
count_df

,Total Player Count
0,576



### Purchasing Analysis (Total)

* Number of Unique Items
* Average Purchase Price
* Total Number of Purchases
* Total Revenue


In [52]:
purch_analysis = {'Number of Unique Items': purchase_data['Item Name'].nunique(),
                  'Average Purchase Price': '${:,.2f}'.format(purchase_data['Price'].mean()),
                  'Total Number of Purchases': purchase_data['Purchase ID'].nunique(),
                  'Total Revenue': '${:,.2f}'.format(purchase_data['Price'].sum())}
#pd.options.display.float_format = 
pa_df=pd.DataFrame([purch_analysis],index=[""])
pa_df.transpose()

,
Average Purchase Price,$3.05
Number of Unique Items,179
Total Number of Purchases,780
Total Revenue,"$2,379.77"


### Gender Demographics

* Percentage and Count of Male Players
* Percentage and Count of Female Players
* Percentage and Count of Other / Non-Disclosed

In [6]:
#filter purchase data by unique users (for later use)
uq_players = purchase_data.drop_duplicates(subset='SN',keep='first').drop(columns='Price')
uq_totals= purchase_data.groupby(['SN'])['Price'].sum()
uq_players = uq_players.merge(uq_totals,how='outer',on='SN').rename(index=str,columns={'Price':'Total Purchase Value'})
uq_players.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Total Purchase Value
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",$10.02
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,$1.56
2,2,Ithergue48,24,Male,92,Final Critic,$4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,$3.27
4,4,Iskosia90,23,Male,131,Fury,$1.44


In [7]:
#function to calculate count and percentages for demographics
def count_and_pc(col,val,total_val):
    val_count=(uq_players[col]==val).sum()
    val_pc="{:.2%}".format(val_count/total_val)
    return [val_count,val_pc]

In [8]:
gender_dict={}
for gender in uq_players['Gender']:
    gender_dict[gender]=count_and_pc('Gender',gender,total_count)
gender_df=pd.DataFrame(gender_dict,index=['Count','Percentage']).transpose()
gender_df

,Count,Percentage
Male,484,84.03%
Other / Non-Disclosed,11,1.91%
Female,81,14.06%


### Purchasing Analysis (Gender)

* The below each broken by gender
  * Purchase Count
  * Average Purchase Price
  * Total Purchase Value
  * Average Purchase Total per Person by Gender

In [9]:
#Purchase Count for each gender
gender_count = purchase_data.groupby(['Gender'])['Price'].count()
#Total purchase value
gender_total=purchase_data.groupby(['Gender'])['Price'].sum()
#Average purchase value by gender
gender_avg = purchase_data.groupby(['Gender'])['Price'].mean()
#Average purchases by user by gender
gender_user_avg = uq_players.groupby(['Gender'])['Total Purchase Value'].mean()


In [86]:
#Purchase count share by gender
purch_share = ["{:.2%}".format(count/gender_count.sum()) for count in gender_count]
#Revenue share by gender - for analysis
rev_share = ["{:.2%}".format(total/gender_total.sum()) for total in gender_total]

In [101]:
summary = {'Purchase Count':gender_count,
           'Purchase Share':purch_share,
          'Average Purchase Price':gender_avg,
          'Total Purchase Value': gender_total,
           'Revenue Share':rev_share,
          'Avg Total Purchase Per Person':gender_user_avg}
gender_summary=pd.DataFrame(summary)
gender_summary

,Purchase Count,Purchase Share,Average Purchase Price,Total Purchase Value,Revenue Share,Avg Total Purchase Per Person
Gender,,,,,,
Female,113,14.49%,$3.20,$361.94,15.21%,$4.47
Male,652,83.59%,$3.02,"$1,967.64",82.68%,$4.07
Other / Non-Disclosed,15,1.92%,$3.35,$50.19,2.11%,$4.56


### Age Demographics

* The below each broken into bins of 4 years (i.e. &lt;10, 10-14, 15-19, etc.)
  * Purchase Count
  * Average Purchase Price
  * Total Purchase Value
  * Average Purchase Total per Person by Age Group

In [13]:
bins=[0,9,14,19,24,29,34,39,44]
labels=['<10','10-14','15-19','20-24','25-29','30-34','35-39','40 and up']
purchase_data['Age Group']=pd.cut(purchase_data['Age'],bins,labels=labels)
uq_players['Age Group']=pd.cut(uq_players['Age'],bins,labels=labels)

In [14]:
#Age Demographics
age_dict={}
for label in labels:
    age_dict[label]=count_and_pc('Age Group',label,total_count)
age_df=pd.DataFrame(age_dict,index=['Count','Percentage']).transpose()
age_df

,Count,Percentage
<10,17,2.95%
10-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
25-29,77,13.37%
30-34,52,9.03%
35-39,31,5.38%
40 and up,11,1.91%


In [15]:
#Purchase Count for each age group
age_count = purchase_data.groupby(['Age Group'])['Price'].count()
#Total purchase value
age_total=purchase_data.groupby(['Age Group'])['Price'].sum()
#Average purchase value by age group
age_avg = purchase_data.groupby(['Age Group'])['Price'].mean()
#Average purchases by user by age group
age_user_avg = uq_players.groupby(['Age Group'])['Total Purchase Value'].mean()

In [91]:
#Purchase Share - for analysis
age_pshare = ["{:.2%}".format(count/age_count.sum()) for count in age_count]
#Revenue Share - for analysis
age_rshare = ["{:.2%}".format(total/age_total.sum()) for total in age_total]

In [94]:
summary = {'Purchase Count':age_count,
           'Purchase Share':age_pshare,
          'Average Purchase Price':age_avg,
          'Total Purchase Value': age_total,
           'Revenue Share': age_rshare,
          'Avg Total Purchase Per Person':age_user_avg}
summary_df=pd.DataFrame(summary)
summary_df

,Purchase Count,Purchase Share,Average Purchase Price,Total Purchase Value,Revenue Share,Avg Total Purchase Per Person
Age Group,,,,,,
<10,23,2.95%,$3.35,$77.13,3.24%,$4.54
10-14,28,3.59%,$2.96,$82.78,3.48%,$3.76
15-19,136,17.46%,$3.04,$412.89,17.36%,$3.86
20-24,365,46.85%,$3.05,"$1,114.06",46.85%,$4.32
25-29,101,12.97%,$2.90,$293.00,12.32%,$3.81
30-34,73,9.37%,$2.93,$214.00,9.00%,$4.12
35-39,41,5.26%,$3.60,$147.67,6.21%,$4.76
40 and up,12,1.54%,$3.04,$36.54,1.54%,$3.32


### Top Spenders

* Identify the the top 5 spenders in the game by total purchase value, then list (in a table):
  * SN
  * Purchase Count
  * Average Purchase Price
  * Total Purchase Value

In [17]:
top_five= uq_players.sort_values(by='Total Purchase Value',ascending=False).head(5)
top5_users=top_five['SN']
top5_counts=[]
top5_avg=[]
top5_totals=top_five['Total Purchase Value']
for i in range(5):
    purchase_count=purchase_data.loc[purchase_data['SN']==top_five.SN[i]]['Price'].count()
    top5_counts.append(purchase_count)
    purchase_avg=purchase_data.loc[purchase_data['SN']==top_five.SN[i]]['Price'].mean()
    top5_avg.append(purchase_avg)

In [18]:
top5_data={'SN':top5_users,'Purchase Count':top5_counts,'Average Purchase Price':purchase_avg,'Total Purchase Value':top5_totals}
top5_df=pd.DataFrame(top5_data)
top5_df

,SN,Purchase Count,Average Purchase Price,Total Purchase Value
72,Lisosia93,5,$4.37,$18.96
253,Idastidru52,4,$4.37,$15.45
201,Chamjask73,3,$4.37,$13.83
120,Iral74,4,$4.37,$13.62
134,Iskadarya95,3,$4.37,$13.10


### Most Popular Items

* Identify the 5 most popular items by purchase count, then list (in a table):
  * Item ID
  * Item Name
  * Purchase Count
  * Item Price
  * Total Purchase Value

In [106]:
uq_items = purchase_data.drop_duplicates(subset='Item Name',keep='first').drop(columns=['Purchase ID','SN','Age','Gender','Age Group'])
item_totals=purchase_data.groupby(['Item ID'])['Price'].sum()
item_count=purchase_data.groupby(['Item ID'])['Price'].count()
uq_items = uq_items.merge(item_totals, on='Item ID').rename(index=str,columns={'Price_x':'Item Price','Price_y':'Total Purchase Value'})
uq_items=uq_items.merge(item_count,how='outer',on='Item ID').rename(index=str,columns={'Price':'Purchase Count'})
uq_items.sort_values(ascending=False,by='Purchase Count').head(10)
#extended to 10 items since many have the same number of purchases

,Item ID,Item Name,Item Price,Total Purchase Value,Purchase Count
24,178,"Oathbreaker, Last Hope of the Breaking Storm",$4.23,$50.76,12
93,145,Fiery Glass Crusader,$4.58,$41.22,9
17,82,Nirvana,$4.90,$44.10,9
0,108,"Extraction, Quickblade Of Trembling Hands",$3.53,$31.77,9
89,75,Brutality Ivory Warmace,$2.42,$19.36,8
57,34,Retribution Axe,$2.22,$17.76,8
28,37,"Shadow Strike, Glory of Ending Hope",$3.16,$25.28,8
2,92,Final Critic,$4.88,$39.04,8
39,72,Winter's Bite,$3.77,$30.16,8
48,60,Wolf,$3.54,$28.32,8


### Most Profitable Items

* Identify the 5 most profitable items by total purchase value, then list (in a table):
  * Item ID
  * Item Name
  * Purchase Count
  * Item Price
  * Total Purchase Value

In [107]:
uq_items.sort_values(ascending=False,by='Total Purchase Value').head(10)
#extended to top 10 to compare with purchase price

,Item ID,Item Name,Item Price,Total Purchase Value,Purchase Count
24,178,"Oathbreaker, Last Hope of the Breaking Storm",$4.23,$50.76,12
17,82,Nirvana,$4.90,$44.10,9
93,145,Fiery Glass Crusader,$4.58,$41.22,9
2,92,Final Critic,$4.88,$39.04,8
111,103,Singed Scalpel,$4.35,$34.80,8
71,59,"Lightning, Etcher of the King",$4.23,$33.84,8
0,108,"Extraction, Quickblade Of Trembling Hands",$3.53,$31.77,9
85,78,"Glimmer, Ender of the Moon",$4.40,$30.80,7
39,72,Winter's Bite,$3.77,$30.16,8
48,60,Wolf,$3.54,$28.32,8


As final considerations:

* You must use the Pandas Library and the Jupyter Notebook.
* You must submit a link to your Jupyter Notebook with the viewable Data Frames.
* You must include a written description of three observable trends based on the data.
* See [Example Solution](HeroesOfPymoli/HeroesOfPymoli_starter.ipynb) for a reference on expected format.

In [103]:
gender_analysis=gender_df.merge(gender_summary, how ="outer", left_index=True, right_on='Gender')
gender_analysis

,Count,Percentage,Purchase Count,Purchase Share,Average Purchase Price,Total Purchase Value,Revenue Share,Avg Total Purchase Per Person
Gender,,,,,,,,
Male,484,84.03%,652,83.59%,$3.02,"$1,967.64",82.68%,$4.07
Other / Non-Disclosed,11,1.91%,15,1.92%,$3.35,$50.19,2.11%,$4.56
Female,81,14.06%,113,14.49%,$3.20,$361.94,15.21%,$4.47


In [99]:
age_analysis=age_df.merge(summary_df, how ="outer", left_index=True, right_on='Age Group')
age_analysis

,Count,Percentage,Purchase Count,Purchase Share,Average Purchase Price,Total Purchase Value,Revenue Share,Avg Total Purchase Per Person
Age Group,,,,,,,,
<10,17,2.95%,23,2.95%,$3.35,$77.13,3.24%,$4.54
10-14,22,3.82%,28,3.59%,$2.96,$82.78,3.48%,$3.76
15-19,107,18.58%,136,17.46%,$3.04,$412.89,17.36%,$3.86
20-24,258,44.79%,365,46.85%,$3.05,"$1,114.06",46.85%,$4.32
25-29,77,13.37%,101,12.97%,$2.90,$293.00,12.32%,$3.81
30-34,52,9.03%,73,9.37%,$2.93,$214.00,9.00%,$4.12
35-39,31,5.38%,41,5.26%,$3.60,$147.67,6.21%,$4.76
40 and up,11,1.91%,12,1.54%,$3.04,$36.54,1.54%,$3.32
